In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Analysis2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Load Amazon Data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
sports_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
sports_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [4]:
# Create the vine_table DataFrame
vine_df = sports_df.select(["review_id", "star_rating", "helpful_votes", "vine", "verified_purchase", "total_votes"])
vine_df.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R1WBPB8MDCCN8F|          5|            0|   N|                N|          0|
|R32M0YEWV77XG8|          5|            1|   N|                Y|          1|
| RR8V7WR27NXJ5|          1|            0|   N|                Y|          0|
|R1MHO5V9Z932AY|          5|            0|   N|                Y|          0|
|R16PD71086BD2V|          5|            0|   N|                N|          1|
|R1Z8IFGWTRWXT6|          3|            0|   N|                Y|          0|
|R3AUMSHAW73HWN|          4|            2|   N|                N|          3|
|R2KWDWFOHGX6FL|          5|            0|   N|                Y|          0|
|R3H9543FWBWFBU|          5|            1|   N|                Y|          1|
| RUANXOQ9W3OU5|          5|            0|   N|                Y

In [10]:
# Filter the data for total_votes > 20
vote_count = vine_df.filter(vine_df["total_votes"] >= 20)

#df3 = df2.filter(df2["total_votes"] >= 20)

In [12]:
new_vote_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
new_vote_table.show()
#df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R1B4753QMTD8ON|          5|           32|   N|                N|         32|
|R28BVHOEL6GCHZ|          5|           21|   N|                Y|         21|
|R27IB4BRKO71I0|          2|           16|   N|                Y|         20|
| RJ86OW12Y4NRL|          5|          124|   N|                Y|        128|
|R3KRABOWCH6ZE4|          1|           48|   N|                Y|         65|
|R1OAKARA30DIWX|          4|           83|   N|                Y|         89|
|R36LSV4WQ4VB1K|          5|           17|   N|                N|         22|
|R18BTOE5UGN7UA|          5|           26|   N|                Y|         28|
|R1DXLGDSK2WMLD|          4|           22|   N|                N|         22|
| RZG2OHQ12YYZI|          5|           30|   N|                Y

In [14]:
# Create a new DF that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
new_vote_table.filter(new_vote_table["vine"] == "Y").show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R32OMS1LSDRCOB|          4|           27|   Y|                N|         31|
|R2IPKK4I0XRCFR|          4|           16|   Y|                N|         21|
| RXYYJ8SGI9IX6|          5|           36|   Y|                N|         41|
|R1SAUBFC7T6JHC|          5|           73|   Y|                N|         75|
|R329HVJKKHWSG3|          5|           59|   Y|                N|         59|
| RZL21K50M52KW|          5|           22|   Y|                N|         22|
|R24RC5J2D92SBE|          5|           19|   Y|                N|         20|
|R1803HITBY7Y08|          4|           92|   Y|                N|        104|
|R1888B8Z6QWYZ4|          4|           18|   Y|                N|         25|
|R1S538NK2I5PRL|          5|           21|   Y|                N

In [16]:
# Create a new DF that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
new_vote_table.filter(new_vote_table["vine"] == "N").show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R1B4753QMTD8ON|          5|           32|   N|                N|         32|
|R28BVHOEL6GCHZ|          5|           21|   N|                Y|         21|
|R27IB4BRKO71I0|          2|           16|   N|                Y|         20|
| RJ86OW12Y4NRL|          5|          124|   N|                Y|        128|
|R3KRABOWCH6ZE4|          1|           48|   N|                Y|         65|
|R1OAKARA30DIWX|          4|           83|   N|                Y|         89|
|R36LSV4WQ4VB1K|          5|           17|   N|                N|         22|
|R18BTOE5UGN7UA|          5|           26|   N|                Y|         28|
|R1DXLGDSK2WMLD|          4|           22|   N|                N|         22|
| RZG2OHQ12YYZI|          5|           30|   N|                Y

In [18]:
# Filter table for five star reviews
five_star = new_vote_table.filter(new_vote_table["star_rating"]== 5)
five_star.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R1B4753QMTD8ON|          5|           32|   N|                N|         32|
|R28BVHOEL6GCHZ|          5|           21|   N|                Y|         21|
| RJ86OW12Y4NRL|          5|          124|   N|                Y|        128|
|R36LSV4WQ4VB1K|          5|           17|   N|                N|         22|
|R18BTOE5UGN7UA|          5|           26|   N|                Y|         28|
| RZG2OHQ12YYZI|          5|           30|   N|                Y|         31|
|R16KLPJUNC9GS7|          5|           20|   N|                Y|         21|
|R1H6DQ1UXL6K8U|          5|           86|   N|                N|         97|
|R1ABJ5P5XAZ9AW|          5|          174|   N|                N|        187|
|R2RRFZ2YFSJ2OF|          5|           27|   N|                Y

In [19]:
# Determine the total number of 5-star reviews
five_star.count()

32804

In [20]:
# Determine the total number of reviews
new_vote_table.count()

61948

In [21]:
# Percentage of five star reviews
five_star.count() / new_vote_table.count()

0.5295409052753922

In [22]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid)
five_star.filter(five_star["verified_purchase"] == "Y").count()/new_vote_table.filter(new_vote_table["verified_purchase"]== "Y").count()

0.5361346516007532